In [4]:
import os 
%pwd

'd:\\ITI\\DS Track\\Deep Learning\\Projects\\Kidney Classification\\Kidney-Disease-Classification\\research'

In [5]:
os.chdir("../")
%pwd

'd:\\ITI\\DS Track\\Deep Learning\\Projects\\Kidney Classification\\Kidney-Disease-Classification'

In [6]:
import dagshub
dagshub.init(repo_owner='omarkhadrawy10', repo_name='Kidney-Disease-Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as omarkhadrawy10

Initialized MLflow to track repo "omarkhadrawy10/Kidney-Disease-Classification"

Repository omarkhadrawy10/Kidney-Disease-Classification initialized!

In [7]:
import torch
model = torch.load(r"D:\ITI\DS Track\Deep Learning\Projects\Kidney Classification\Kidney-Disease-Classification\artifacts\training\model.pth",weights_only=False)
model

d:\ITI\DS Track\Deep Learning\Projects\Kidney Classification\Kidney-Disease-Classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


InceptionResnetV2(
  (conv2d_1a): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (conv2d_2a): ConvNormAct(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (conv2d_2b): ConvNormAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): ConvNormAct(
    (conv): Conv2d(64, 80, kernel_size=(1, 

# Entity

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params:dict
    mlflow_uri: str
    params_image_size:list
    params_batch_size:int

    

# config


In [9]:
from src.CNNClassifierKidneyDiseases.constants import *
from src.CNNClassifierKidneyDiseases.utils.common import read_yaml , create_directories,save_json



In [10]:
class ConfigurationManager:
    def __init__ (self, config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(

            path_of_model="artifacts/training/model.pth",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/omarkhadrawy10/Kidney-Disease-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

            
        )

        return eval_config
    

# Components

In [11]:
import torch
from pathlib import Path
import mlflow
import mlflow.pytorch
from urllib.parse import urlparse


In [16]:
import mlflow.pytorch
import mlflow.pytorch
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import mlflow
import mlflow.pytorch
from urllib.parse import urlparse
from pathlib import Path
from src.CNNClassifierKidneyDiseases.utils.common import save_json
#from src.CNNClassifierKidneyDiseases.entity.config_entity import EvaluationConfig

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_loader(self):
        transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[1:]),
            transforms.ToTensor()
        ])

        full_dataset = datasets.ImageFolder(root=self.config.training_data, transform=transform)

        # Split dataset into train and validation
        val_size = int(0.3 * len(full_dataset))
        train_size = len(full_dataset) - val_size
        _, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

        self.valid_loader = DataLoader(
            val_dataset,
            batch_size=self.config.params_batch_size,
            shuffle=False
        )

    @staticmethod
    def load_model(path: Path):
        return torch.load(path , weights_only=False)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.model.eval()

        self._valid_loader()

        correct = 0
        total = 0
        loss_total = 0.0
        criterion = torch.nn.CrossEntropyLoss()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)

        with torch.no_grad():
            for images, labels in self.valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss_total += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        avg_loss = loss_total / total
        self.score = (avg_loss, accuracy)

        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        #mlflow.set_tracking_uri(self.config.mlflow_uri)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            if tracking_url_type_store != "file":
                mlflow.pytorch.log_model(self.model, artifact_path ="model")
            else:
                mlflow.pytorch.log_model(self.model, artifact_path = "model")


# Pipeline

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e :
    raise e

[2025-07-07 12:26:20,298: INFO: common: yaml file config\config.yaml loaded successfully]
[2025-07-07 12:26:20,300: INFO: common: yaml file params.yaml loaded successfully]
[2025-07-07 12:26:20,302: INFO: common: Created Directory at : artifacts]
[2025-07-07 12:26:35,267: INFO: common: json File Saved Successfully at: scores.json]


2025/07/07 12:26:43 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


[2025-07-07 12:28:19,805: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')': /omarkhadrawy10/Kidney-Disease-Classification.mlflow/api/2.0/mlflow-artifacts/artifacts/5419a5f0491845e2a1d40e4135c6e3eb/49cb6201ee3e468f9f2030670c38b2ad/artifacts/model/data/model.pth]


In [ ]:
import mlflow
print(mlflow.__version__)

1.27.0


In [ ]:
import dagshub 
print(dagshub.__version__)

0.5.10
